In [37]:
import os 
import pandas as pd


In [38]:
vector_data=pd.read_csv('D:\Model_Deployement\End_to_End_Phishing_mail_detection_ML_project\\artifacts\data_preprocessed\\vectorized_data.csv',index_col=False)

In [39]:
k=vector_data['Email_vector']

In [40]:
import numpy as np
def convert_string_to_array(vector_string):
    """Convert string representation of vector to numpy array"""
    try:
        # Remove brackets and split by whitespace
        vector_string = vector_string.strip('[]')
        # Convert to numpy array
        return np.fromstring(vector_string, sep=' ')
    except:
        return None

vector_data['Email_vector'] = vector_data['Email_vector'].apply(convert_string_to_array)

    
 # Example usage, convert the first vector string to numpy array

In [42]:
vector_data['Email Type']


0        0
1        0
2        0
3        1
4        1
        ..
18645    1
18646    0
18647    0
18648    0
18649    1
Name: Email Type, Length: 18650, dtype: int64

In [44]:
vector_data=vector_data[['Email_vector','Email Type']]

In [46]:
vector_data.rename(columns={'Email_vector':'Feature','Email Type':'Target'},inplace=True)

In [47]:
vector_data.head()

,Feature,Target
0,"[-0.63010776, 0.34649888, 0.16577671, 0.050873...",0
1,"[-0.6872441, 0.8160633, 0.26467782, 0.01559338...",0
2,"[-1.1524391, 1.2123232, -0.79381204, 1.2315739...",0
3,"[-0.622395635, 0.284755796, -0.675644875, 0.20...",1
4,"[-0.6398096, 0.47042787, -0.0013294, 0.3611383...",1


In [ ]:
from sklearn.model_selection import train_test_split
X = vector_data['Feature'].values
y = vector_data['Target'].values


# Model training on the data

In [13]:
import os
from pathlib import Path


In [25]:
os.getcwd()

'd:\\Model_Deployement\\End_to_End_Phishing_mail_detection_ML_project'

In [10]:
os.chdir('../')

In [31]:
from dataclasses import dataclass
@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir :Path
    training_data_path: Path
    trained_model_dir: Path
    # model_name: str
    # model_params: dict


In [32]:
from Phising_mail_detection.constants import *
from Phising_mail_detection.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_preprocessing_config(self) -> ModelTrainingConfig:
        config = self.config.Model_training
        # params=self.params.model_params
        # schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        Model_train_config = ModelTrainingConfig(
            root_dir = config.root_dir,
            training_data_path= config.training_data_path,
            trained_model_dir= config.trained_model_dir,
            # model_name = params.model_name,
            # model_params = params.model_params
        )

        return Model_train_config


In [35]:
from Phising_mail_detection import logger
import numpy as np
from Phising_mail_detection.utils.common import load_model,save_model
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,classification_report
from lightgbm import LGBMClassifier


class Model_training_function:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config
    
    def convert_string_to_array(self,vector_string):
        
      """Convert string representation of vector to numpy array"""
      try:
        # Remove brackets and split by whitespace
            vector_string = vector_string.strip('[]')
        # Convert to numpy array
            return np.fromstring(vector_string, sep=' ')
      except Exception as e:
            return e
      

    def data_preparation(self):
        logger.info("Loading training data")
        training_data = pd.read_csv(self.config.training_data_path)
        training_data['Email_vector'] = training_data['Email_vector'].apply(self.convert_string_to_array)
        logger.info("Splitting data into features and target")
        X = training_data['Email_vector']
        y = training_data['Email Type'].values
        logger.info("Splitting data into train and test sets")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        X_train = np.vstack(X_train)
        X_test= np.vstack(X_test)
        print(X_train.shape)
        return X_train, X_test, y_train, y_test
    
    def model_training(self):
        logger.info("Training the model")
        
        X_train, X_test, y_train, y_test = self.data_preparation()
                # model = LGBMClassifier()
        # model = LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
        model=RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)
        logger.info("Model evaluation on training data")
        y_train_pred = model.predict(X_train)
        train_accuracy = accuracy_score(y_train, y_train_pred)
        train_classification_rep = classification_report(y_train, y_train_pred)
        print(f"Training Accuracy: {train_accuracy}")
        print("Training Classification Report:")
        print(train_classification_rep)
        logger.info("Evaluating the model")
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        classification_rep = classification_report(y_test, y_pred)
        print(f"Accuracy: {accuracy}")
        print("Classification Report:")
        print(classification_rep)
        print("Confusion Matrix:")
        print(confusion_matrix(y_test, y_pred))
        logger.info("Saving the trained model")
        save_model(model, self.config.trained_model_dir)


In [36]:
try:
    config = ConfigurationManager()
    model_training_config = config.get_data_preprocessing_config()
    model= Model_training_function(config=model_training_config)
    model.model_training()
except Exception as e:
    logger.exception(e)
    raise e

[2025-08-05 14:07:56,520:'INFO':common:yaml file: config\config.yaml loaded successfully]
[2025-08-05 14:07:56,524:'INFO':common:yaml file: params.yaml loaded successfully]
[2025-08-05 14:07:56,524:'INFO':common:yaml file: schema.yaml loaded successfully]
[2025-08-05 14:07:56,530:'INFO':common:created directory at: artifacts]
[2025-08-05 14:07:56,530:'INFO':common:created directory at: artifacts/model_training]
[2025-08-05 14:07:56,530:'INFO':1355998675:Training the model]
[2025-08-05 14:07:56,538:'INFO':1355998675:Loading training data]
[2025-08-05 14:08:00,503:'INFO':1355998675:Splitting data into features and target]
[2025-08-05 14:08:00,503:'INFO':1355998675:Splitting data into train and test sets]
(14920, 100)
[2025-08-05 14:08:29,193:'INFO':1355998675:Model evaluation on training data]
Training Accuracy: 0.988941018766756
Training Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      9049
           1       